Apertura del file

In [1]:
import json

with open('clef2024-checkthat-lab-main-task5/task5/data/English_dev.json', 'r') as f:
    data = json.load(f)


Suddivisione in train e test

In [2]:
train = data[:-1]
test = data[-1]

Adesso vado a vettorizzare il training 

Prima di tutto creo una funzione che prenda un oggetto (rumor) e restutuisce i primi 5 tweet più significativi

Faccio restituire anche la label

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import heapq
import re

def flat_list(lista):
    ris = []
    for elemento in lista.toarray():
        ris.extend(elemento)
    return ris

def vettorizza(tweet):
    pass

def tweetSignificativi(rumor):
    tweet = []
    pattern = r"\bhttps?.*\b"
    patternHashtag = r"\b\#(.*)\b"
    tweet.append(re.sub(pattern, '', rumor['rumor']))

    #Pulizia
    for autority in rumor['timeline']:
        #Regex per la rimozione dei link: \Whttps?.*\W
        correctString = re.sub(pattern, '', autority[2])
        #Regex per la rimozione degli hashtag
        #correctString = re.sub(patternHashtag, '', autority[2])
        tweet.append(correctString)

    #Calcolo coseno
    vectorizer = TfidfVectorizer(use_idf=True,sublinear_tf=True,smooth_idf=True, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(tweet)

    #Vado ad "appiattire" la matrice
    cosine_similarities = []
    for riga in tfidf_matrix:
        cosine_similarities.append(cosine(flat_list(tfidf_matrix[0]),flat_list(riga)))

    #Recupero dei cinque tweet più simili
    smallest_five = heapq.nsmallest(6, cosine_similarities)

    their_indices = [i for i, value in enumerate(cosine_similarities) if value in smallest_five]
    ris = []
    for index in their_indices:
        ris.append((rumor['label'], tweet[index]))

    return ris

Vado a creare un mega listone di tutti i tweet significativi

In [24]:
listaTS = []
for rumor in train:
    listaTS.extend(tweetSignificativi(rumor))

/opt/anaconda3/lib/python3.11/site-packages/scipy/spatial/distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ora abbiamo i tweet più significativi etichettati

Quello che devo fare adesso è andare a dividere in x e y per l'addestramento

creo la funzione per dividere i vettori

In [27]:
def dividiVettori(vettore):
    X = []
    Y = []
    for elemento in vettore:
        X.append(elemento[0])
        Y.append(elemento[1])
    
    return X,Y

Controllo la suddivisione

In [57]:
Y,X = dividiVettori(listaTS)

Vado a calcolare la vettorizzazione

Aggiungo anche il test alla vettorizzazione, per avere lo stesso numero di features

In [59]:
tst = tweetSignificativi(test)
Yt,Xt = dividiVettori(tst)

In [61]:
X

['Naturalization decree in preparation: Lebanese passports for sale?! ',
 'President Aoun received the Lebanese Ambassador to Canada, Ambassador Fadi Ziadeh, who briefed him on the conditions of the Lebanese community in Canada, Lebanese-Canadian relations, and ways to develop them in all fields. ',
 'The President of the Republic awarded the Papal Ambassador the National Order of the Cedar, with the rank of Grand Officer, wishing him success and success in his new mission: Lebanon always finds the Holy See by its side in happy circumstances as well as in the difficult circumstances it is going through, and this is a reason for gratitude and thanks from all the Lebanese ',
 '“The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news.',
 'President Aoun received the Ambassador of Finland, Tarja Fernandes, on a farewell visit on the occasion of the 

In [62]:
X.extend(Xt)

In [63]:
X

['Naturalization decree in preparation: Lebanese passports for sale?! ',
 'President Aoun received the Lebanese Ambassador to Canada, Ambassador Fadi Ziadeh, who briefed him on the conditions of the Lebanese community in Canada, Lebanese-Canadian relations, and ways to develop them in all fields. ',
 'The President of the Republic awarded the Papal Ambassador the National Order of the Cedar, with the rank of Grand Officer, wishing him success and success in his new mission: Lebanon always finds the Holy See by its side in happy circumstances as well as in the difficult circumstances it is going through, and this is a reason for gratitude and thanks from all the Lebanese ',
 '“The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news.',
 'President Aoun received the Ambassador of Finland, Tarja Fernandes, on a farewell visit on the occasion of the 

In [64]:

vectorizer = TfidfVectorizer(use_idf=True,sublinear_tf=True,smooth_idf=True, stop_words=None)
tfidf_matrix = vectorizer.fit_transform(X)

X = tfidf_matrix[:-(len(Xt))]
Xt = tfidf_matrix[-(len(Xt)):]

In [66]:
Xt

<6x1822 sparse matrix of type '<class 'numpy.float64'>'
	with 138 stored elements in Compressed Sparse Row format>

Addestramento del modello

In [68]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()

model= NB.fit(X,Y)

Adesso vettorizzazione del test e previsione

Devo estrarre quelli più significativi e vettorizzarli

In [69]:
model.predict(Xt)

array(['REFUTES', 'REFUTES', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO',
       'NOT ENOUGH INFO', 'NOT ENOUGH INFO'], dtype='<U15')